In [ ]:
import iris
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patch
import pandas as pd
import numpy as np
import os
import glob


import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib as mpl

from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


import sys
sys.path.insert(0,'/nfs/mary/Users/ee19b2s/work/chapter1/hs1_pmip_proj/classes/')
from simulation_dictionary import tejen, tejeo, deglg, deglh, loveclim, iloveclim, iloveclim_fwf, iloveclim_uniform, iloveclim_fwf_glac, iloveclim_uniform_glac, famous, itrace, trace, mpi08, mpi10, mpi11, mpi12, miroc, uvic_short, uvic_long
#import data_SAT_3D_dictionary as ddict_sat
import variable_dictionary as vdict
sys.path.insert(0,'/nfs/mary/Users/ee19b2s/work/chapter1/hs1_pmip_proj/modules/')
import hadcm3_functions as hf 
import loveclim_functions as lf
import itrace_functions as itf
import functions_for_comparisons as func4comp
import matplotlib.gridspec as gridspec
from regress_func import regress

from functions_for_comparisons import find_nearest_value
import scipy.stats as stats

lon = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/1degree_lon.npy')
lat = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/1degree_lat.npy')
expt_order_main = [tejen, loveclim, itrace, trace, deglg, mpi10, iloveclim_uniform, iloveclim_uniform_glac, deglh, mpi11, mpi12, iloveclim_fwf, iloveclim_fwf_glac, famous, miroc]
expt_order_main_wUvic = [tejen, loveclim, itrace, trace, deglg, mpi10, iloveclim_uniform, iloveclim_uniform_glac, deglh, mpi11, mpi12, iloveclim_fwf, iloveclim_fwf_glac, famous, miroc, uvic_short, uvic_long]

# fig05 and figS03: onset of deglaciation

### calculation of the data for this figure is performed with roche_analysis_ttest_unequal_samples.py

In [ ]:
class MidpointNormalize(mpl.colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        # Note also that we must extrapolate beyond vmin/vmax
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1.]
        return np.ma.masked_array(np.interp(value, x, y,
                                            left=-np.inf, right=np.inf))

    def inverse(self, value):
        y, x = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y, left=-np.inf, right=np.inf)

    
normalize = MidpointNormalize(vcenter=18, vmin=13, vmax=20.5)



In [ ]:

### COLLECT FILES FROM LOCATION

out_times_all = []
files = []
for file in glob.glob('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/roche_analysis_new_for_review/*.roche_analysis_ttest_greater_SAT_99conf_500yrLGM.100yr.npy'):
    files.append(file)
files.sort()  
for expt in expt_order_main:
    index = [i for i, s in enumerate(files) if '/'+expt['expt_name']+'.' in s]
    index = index[0]
    out_times_all.append(np.load(files[index]))
    

### DETERMINE LOCATIONS FOR HATCHING
    
mask_all = []
for file in out_times_all:
    mask = np.zeros((file.shape[0], file.shape[1]))
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if np.isnan(file[i,j]):
                mask[i,j] = 0
            else:
                mask[i,j] = np.nan
    mask_all.append(mask)
    


### CALCULATE MEAN AND VARIANCE

out_times_mean = np.nanmean(out_times_all, axis=0)
out_times_var = np.nanvar(out_times_all, axis=0)



### PUT ALL DATA IN LIST WITH EXPTS

#expts=[deglg,deglh,famous,iloveclim_fwf,iloveclim_uniform,iloveclim_fwf_glac,iloveclim_uniform_glac,itrace,loveclim,miroc,mpi10,mpi11,mpi12,tejen,trace]
all_data = out_times_all
all_data.append(out_times_mean)
all_data.append(out_times_var)



### PLOT ALL THE DATA 

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

fig,axgr = plt.subplots(nrows=6, ncols=3, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
plt.gcf().set_size_inches(7,8)


#axgr[-2].set_title('mean', fontsize=40)
#axgr[-1].set_title('variance', fontsize=40)
axgr = axgr.flat
axgr[-3].set_title('mean', fontsize=8)
axgr[-2].set_title('variance', fontsize=8)
for ax,name in zip(axgr, expt_order_main):
    ax.set_title(name['label'], fontsize=8)


for i,ax in enumerate(axgr[0:17]):
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

    data,lons = add_cyclic_point(all_data[i], coord=lon)
    
    
    if i == len(axgr)-2:
        contour = ax.contourf(lons, lat, data, levels=20, transform=projection, cmap='PuRd') #YlOrRd')
    else:
        contour = ax.contourf(lons, lat, data, levels=20, transform=projection, cmap='cubehelix', norm=normalize)
    
    if i < len(axgr)-3:
        mask,lons2 = add_cyclic_point(mask_all[i], coord=lon)
        
        hatch = ax.contourf(lons2, lat, mask, 1, transform=ccrs.PlateCarree(), colors='none', zorder=500, hatches=['//'])#, alpha=0.1)

        for collection in hatch.collections:
            collection.set_edgecolor('black')
            #collection.set_linewidth(0.)

    if i == 13:
        cb_ax = fig.add_axes([0.7, -0.02, .0175, 0.18])
        ticks = [13, 14, 15, 16, 17, 18, 19, 20, 21]
        cbar = fig.colorbar(contour,cax=cb_ax, orientation='vertical',ticks=ticks)
        cbar.ax.tick_params(labelsize=6)
        cbar.set_label('ka BP', fontsize=8)
        cbar.outline.set_linewidth(0.5)
cb_ax = fig.add_axes([.8, -0.02, .0175, 0.18])
ticks = [0, 1, 2, 3, 4, 5, 6]
cbar = fig.colorbar(contour,cax=cb_ax, orientation='vertical')#, ticks=ticks)
cbar.ax.tick_params(labelsize=6)
cbar.set_label('variance', fontsize=8)
cbar.outline.set_linewidth(0.5)

for ax in axgr:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.2)


fig.delaxes(axgr[-1])

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_supplementary_figures_pdfs/figS03_onset_of_deglac_lateLGM.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [ ]:
expt_order_earlyLGM = [trace, deglg, mpi10, iloveclim_uniform, iloveclim_uniform_glac, deglh, mpi11, mpi12, iloveclim_fwf, iloveclim_fwf_glac, famous, miroc]
### COLLECT FILES FROM LOCATION

out_times_all = []
files = []
for file in glob.glob('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/roche_analysis_new_for_review/*.roche_analysis_ttest_greater_SAT_99conf_500yr_earlyLGM.100yr.npy'):
    files.append(file)
files.sort()  
for expt in expt_order_earlyLGM:
    indices = [i for i, s in enumerate(files) if '/'+expt['expt_name']+'.' in s]
    index = indices[0]
    out_times_all.append(np.load(files[index]))
    

### DETERMINE LOCATIONS FOR HATCHING
    
mask_all = []
for file in out_times_all:
    mask = np.zeros((file.shape[0], file.shape[1]))
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if np.isnan(file[i,j]):
                mask[i,j] = 0
            else:
                mask[i,j] = np.nan
    mask_all.append(mask)
    


### CALCULATE MEAN AND VARIANCE

out_times_mean = np.nanmean(out_times_all, axis=0)
out_times_var = np.nanvar(out_times_all, axis=0)



### PUT ALL DATA IN LIST WITH EXPTS

#expts=[deglg,deglh,famous,iloveclim_fwf,iloveclim_uniform,iloveclim_fwf_glac,iloveclim_uniform_glac,itrace,loveclim,miroc,mpi10,mpi11,mpi12,tejen,trace]
all_data = out_times_all
all_data.append(out_times_mean)
all_data.append(out_times_var)



### PLOT ALL THE DATA 

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

fig,axgr = plt.subplots(nrows=5, ncols=3, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
plt.gcf().set_size_inches(20,20)


#axgr[-2].set_title('mean', fontsize=40)
#axgr[-1].set_title('variance', fontsize=40)
axgr = axgr.flat
axgr[-3].set_title('mean', fontsize=25)
axgr[-2].set_title('variance', fontsize=25)
for ax,name in zip(axgr, expt_order_earlyLGM):
    ax.set_title(name['label'], fontsize=25)


for i,ax in enumerate(axgr[0:-1]):
    ax.coastlines()

    data,lons = add_cyclic_point(all_data[i], coord=lon)
    
    
    if i == len(axgr)-2:
        contour = ax.contourf(lons, lat, data, levels=20, transform=projection, cmap='PuRd') #YlOrRd')
    else:
        contour = ax.contourf(lons, lat, data, levels=20, transform=projection, cmap='cubehelix', norm=normalize)
    
    if i < len(axgr)-3:
        mask,lons2 = add_cyclic_point(mask_all[i], coord=lon)
        
        hatch = ax.contourf(lons2, lat, mask, 1, transform=ccrs.PlateCarree(), colors='none', zorder=500, hatches=['//'])#, alpha=0.1)

        for collection in hatch.collections:
            collection.set_edgecolor('black')
            #collection.set_linewidth(0.)

    if i == 0:
        cb_ax = fig.add_axes([0.7, -0.01, .0175, 0.185])
        ticks = [13, 14, 15, 16, 17, 18, 19, 20, 21]
        cbar = fig.colorbar(contour,cax=cb_ax, orientation='vertical',ticks=ticks)
        cbar.ax.tick_params(labelsize=20)
        cbar.set_label('ka BP', fontsize=25)
cb_ax = fig.add_axes([.8, -0.01, .0175, 0.185])
ticks = [0, 1, 2, 3, 4, 5, 6]
cbar = fig.colorbar(contour,cax=cb_ax, orientation='vertical')#, ticks=ticks)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('variance', fontsize=25)
fig.delaxes(axgr[-1])

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_figures_pdfs/fig05_onset_of_deglac_earlyLGM.pdf', format='pdf', dpi=300, bbox_inches='tight')

# fig07: spatial correction AMOC v SAT

In [ ]:
lgm_def = 20
end_def = 15
ddict = ddict_sat
expts_data = [ddict.tejen, ddict.loveclim, ddict.itrace, ddict.trace, ddict.deglg, ddict.mpi10, ddict.iloveclim_uniform, ddict.iloveclim_uniform_glac, ddict.deglh, ddict.mpi11, ddict.mpi12, ddict.iloveclim_fwf, ddict.iloveclim_fwf_glac, ddict.famous, ddict.miroc]

test_ysAMOC = []
test_xsAMOC = []
for expt,data in zip(expt_order_main,expts_data):
    lgm_index = hf.find_nearest_value(expt['time10'], lgm_def)
    end_index = hf.find_nearest_value(expt['time10'], end_def)
    try:             
        test_ysAMOC.append(data['global_3D_temp10'][lgm_index:end_index,0,:,:])
    except:
        test_ysAMOC.append(data['global_3D_temp10'][lgm_index:end_index])

    test_xsAMOC.append(expt['NH_max_amoc10'][lgm_index:end_index])
    
    
times = ['19-18', '18-17', '17-16', '16-15', '15-14']
all_AMOC_Rs = []
all_AMOC_slopes = []
for expt,test_x,test_y in zip(expt_order_main,test_xsAMOC,test_ysAMOC):
    
    r_sq,slope = regress(test_x, test_y)
    
    all_AMOC_Rs.append(r_sq)
    all_AMOC_slopes.append(slope)
    

all_AMOC_Rs_array = np.array(all_AMOC_Rs)
all_AMOC_slopes_array = np.array(all_AMOC_slopes)

np.save('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_Rs.npy', all_AMOC_Rs_array)
np.save('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_slopes.npy', all_AMOC_slopes_array)


In [ ]:
all_AMOC_Rs = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_Rs.npy')
all_AMOC_slopes = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_slopes.npy')

for file in all_AMOC_slopes:
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if file[i,j] > 100 or file[i,j] < -100:
                file[i,j] = 0
            else:
                pass


In [ ]:
#all_AMOC_Rs = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_Rs.npy')
#all_AMOC_slopes = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_AMOC_slopes.npy')

orig_map=plt.cm.get_cmap('cubehelix')
reversed_map = orig_map.reversed()

orig_map2=plt.cm.get_cmap('RdBu')
reversed_map2 = orig_map2.reversed()

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

fig,axgr = plt.subplots(ncols=4, nrows=8, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
plt.gcf().set_size_inches(55,65)

axgr = axgr.flat

#for ax,expt in zip(axgr, expt_order_main):
 #   ax.set_title(expt['label'], fontsize=50)

axgr[0].text(0.45, 1.3, '(a)', transform=axgr[0].transAxes, fontsize=90)
axgr[1].text(0.45, 1.3, '(b)', transform=axgr[1].transAxes, fontsize=90)
axgr[2].text(0.45, 1.3, '(c)', transform=axgr[2].transAxes, fontsize=90)
axgr[3].text(0.45, 1.3, '(d)', transform=axgr[3].transAxes, fontsize=90)

counter = 0
i = 0
j = 0
for ax in axgr[0:30]:
    
    if counter%2 == 0:
        

        ax.coastlines()
        ax.set_title(expt_order_main[i]['label'], fontsize=60)
        data,lons = add_cyclic_point(np.clip(all_AMOC_Rs[i],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=40, transform=projection, cmap=reversed_map, vmin=0, vmax=1)
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour = contour
        
        i +=1
    else:
        
        ax.coastlines()
        data,lons = add_cyclic_point(np.clip(all_AMOC_slopes[j],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=40, transform=projection, cmap=reversed_map2, vmin=-1, vmax=1, norm=mpl.colors.CenteredNorm())
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if j == 3:
            cbar_contour_slope = contour
        j += 1
        
    counter += 1

cb_ax = fig.add_axes([1.02, 0, .017, 0.185])
ticks = [0, .2, .4, .6, .8, 1]
#ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
cbar = fig.colorbar(cbar_contour,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=40)
cbar.set_label('R_squared value', fontsize=50)
fig.delaxes(axgr[-1])
fig.delaxes(axgr[-2])





    
    

cb_ax = fig.add_axes([1.08, 0, .017, 0.185])
ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
#ticks = [-.3,  -.2,  -.1,  0,  .1,  .2, .3, .4]
cbar = fig.colorbar(cbar_contour_slope,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=40)
cbar.set_label('slope (Sv/$^\circ$C)', fontsize=50)
#fig.delaxes(axgr[-1])'''

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_figures_pdfs/fig07_Rs_and_slopes_AMOC_vs_SAT.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [ ]:
orig_map=plt.cm.get_cmap('cubehelix')
reversed_map = orig_map.reversed()

orig_map2=plt.cm.get_cmap('RdBu')
reversed_map2 = orig_map2.reversed()


projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

fig,axgr = plt.subplots(ncols=4, nrows=8, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
#plt.gcf().set_size_inches(55,65)
plt.gcf().set_size_inches(12,14)
axgr = axgr.flat

#for ax,expt in zip(axgr, expt_order_main):
 #   ax.set_title(expt['label'], fontsize=50)

axgr[0].text(0.45, 1.3, '(a)', transform=axgr[0].transAxes, fontsize=20)
axgr[1].text(0.45, 1.3, '(b)', transform=axgr[1].transAxes, fontsize=20)
axgr[2].text(0.45, 1.3, '(c)', transform=axgr[2].transAxes, fontsize=20)
axgr[3].text(0.45, 1.3, '(d)', transform=axgr[3].transAxes, fontsize=20)

counter = 0
i = 0
j = 0
for ax in axgr[0:30]:
    
    if counter%2 == 0:
        

        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.set_title(expt_order_main[i]['label'], fontsize=15)
        data,lons = add_cyclic_point(np.clip(all_AMOC_Rs[i],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=10, transform=projection, cmap=reversed_map, vmin=0, vmax=1)
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour = contour
        
        i +=1
    else:
        
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        data,lons = add_cyclic_point(np.clip(all_AMOC_slopes[j],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=10, transform=projection, cmap=reversed_map2, vmin=-1, vmax=1, norm=mpl.colors.CenteredNorm())
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour_slope = contour
        j += 1
        
    counter += 1

cb_ax = fig.add_axes([1.0, 0, .017, 0.185])
ticks = [0, .2, .4, .6, .8, 1]
#ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
cbar = fig.colorbar(cbar_contour,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=10)
cbar.set_label('R_squared value', fontsize=12)
fig.delaxes(axgr[-1])
fig.delaxes(axgr[-2])





    
    

cb_ax = fig.add_axes([1.08, 0, .017, 0.185])
ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
#ticks = [-.3,  -.2,  -.1,  0,  .1,  .2, .3, .4]
cbar = fig.colorbar(cbar_contour_slope,cax=cb_ax, orientation='vertical')#, ticks=ticks)
cbar.ax.tick_params(labelsize=10)
cbar.set_label('slope (Sv/$^\circ$C)', fontsize=12)
#fig.delaxes(axgr[-1])'''

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_figures_pdfs/fig07_Rs_and_slopes_AMOC_vs_SAT_xsmall.pdf', format='pdf', dpi=300, bbox_inches='tight')

# fig08: spatial correction CO2 v SAT

In [ ]:
lgm_def = 20
end_def = 15
ddict = ddict_sat
expts_data = [ddict.tejen, ddict.loveclim, ddict.itrace, ddict.trace, ddict.deglg, ddict.mpi10, ddict.iloveclim_uniform, ddict.iloveclim_uniform_glac, ddict.deglh, ddict.mpi11, ddict.mpi12, ddict.iloveclim_fwf, ddict.iloveclim_fwf_glac, ddict.famous, ddict.miroc]

test_ysCO2 = []
test_xsCO2 = []
for expt,data in zip(expt_order_main,expts_data):   
    lgm_index = hf.find_nearest_value(expt['time10'], lgm_def)
    end_index = hf.find_nearest_value(expt['time10'], end_def)
    #test_ysCO2.append(split_run_by_index(data['global_3D_temp10'], expt['time10'], sample_time=5000, lgm_def=lgm_def, end_time=end_time))

    try:
        test_ysCO2.append(data['global_3D_temp10'][lgm_index:end_index,0,:,:])
    except:
        test_ysCO2.append(data['global_3D_temp10'][lgm_index:end_index])     
    
    time10 = []
    for i in range(len(expt['co2_type']['time(ka)'])):
        if i%10 == 0:
            time10.append(expt['co2_type']['time(ka)'][i])
    time10 = np.array(time10)
    if 'trace' == expt['expt_name']:
        time10 = expt['co2_type']['time(ka)']
    
    lgm_index = hf.find_nearest_value(time10, lgm_def)
    end_index = hf.find_nearest_value(time10, end_def)

    co2 = np.array(hf.means_10yr_calc(expt['co2_type']['co2']))
    if 'trace' == expt['expt_name']:
        co2 = expt['co2_type']['co2']

    test_xsCO2.append(co2[lgm_index:end_index])
    
    
times = ['19-18', '18-17', '17-16', '16-15', '15-14']
all_co2_Rs = []
all_co2_slopes = []

for expt,test_x,test_y in zip(expt_order_main,test_xsCO2,test_ysCO2):
    
    r_sq,slope = regress(np.array(test_x), test_y)
    
    all_co2_Rs.append(r_sq)
    all_co2_slopes.append(slope)

    
all_co2_Rs_array = np.array(all_co2_Rs)
all_co2_slopes_array = np.array(all_co2_slopes_arrray)

np.save('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_Rs.npy', all_co2_Rs_array)
np.save('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_slopes.npy', all_co2_slopes)
   


In [ ]:
all_co2_Rs = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_Rs.npy')
all_co2_slopes = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_slopes.npy')

for file in all_co2_slopes:
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if file[i,j] > 100 or file[i,j] < -100:
                file[i,j] = 0
            else:
                pass

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

orig_map2=plt.cm.get_cmap('RdBu')
reversed_map2 = orig_map2.reversed()

fig,axgr = plt.subplots(ncols=4, nrows=8, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
plt.gcf().set_size_inches(55,65)

axgr = axgr.flat

#for ax,expt in zip(axgr, expt_order_main):
 #   ax.set_title(expt['label'], fontsize=50)
axgr[0].text(0.45, 1.3, '(a)', transform=axgr[0].transAxes, fontsize=90)
axgr[1].text(0.45, 1.3, '(b)', transform=axgr[1].transAxes, fontsize=90)
axgr[2].text(0.45, 1.3, '(c)', transform=axgr[2].transAxes, fontsize=90)
axgr[3].text(0.45, 1.3, '(d)', transform=axgr[3].transAxes, fontsize=90)


counter = 0
i = 0
j = 0
for ax in axgr[0:30]:
    
    if counter%2 == 0:
        

        ax.coastlines()
        ax.set_title(expt_order_main[i]['label'], fontsize=60)
        data,lons = add_cyclic_point(np.clip(all_co2_Rs[i],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=40, transform=projection, cmap=reversed_map, vmin=0, vmax=1)
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour = contour
        
        i +=1
    else:
        
        ax.coastlines()
        data,lons = add_cyclic_point(np.clip(all_co2_slopes[j],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=40, transform=projection, cmap=reversed_map2, vmin=-1, vmax=1, norm=mpl.colors.CenteredNorm())
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour_slope = contour
        j += 1
        
    counter += 1

cb_ax = fig.add_axes([1.02, 0, .017, 0.185])
ticks = [0, .2, .4, .6, .8, 1]
#ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
cbar = fig.colorbar(cbar_contour,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=40)
cbar.set_label('R_squared value', fontsize=50)
fig.delaxes(axgr[-1])
fig.delaxes(axgr[-2])





    
    

cb_ax = fig.add_axes([1.08, 0, .017, 0.185])
ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
#ticks = [-.3,  -.2,  -.1,  0,  .1,  .2, .3, .4]
cbar = fig.colorbar(cbar_contour_slope,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=40)
cbar.set_label('slope (ppm/$^\circ$C)', fontsize=50)
#fig.delaxes(axgr[-1])'''

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_figures_pdfs/fig08_Rs_and_slopes_CO2_vs_SAT.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [ ]:
all_co2_Rs = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_Rs.npy')
all_co2_slopes = np.load('/nfs/mary/Users/ee19b2s/work/data/pmip_proj_hs1/Rs_and_slopes/all_co2_slopes.npy')

for file in all_co2_slopes:
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if file[i,j] > 100 or file[i,j] < -100:
                file[i,j] = 0
            else:
                pass


projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_project=projection))

fig,axgr = plt.subplots(ncols=4, nrows=8, constrained_layout=True, dpi=200, subplot_kw={'projection':projection})#, gridspec_kw = {'hspace': 0.05, 'wspace': 0.05})
plt.gcf().set_size_inches(12,14)

axgr = axgr.flat

#for ax,expt in zip(axgr, expt_order_main):
 #   ax.set_title(expt['label'], fontsize=50)
axgr[0].text(0.45, 1.3, '(a)', transform=axgr[0].transAxes, fontsize=20)
axgr[1].text(0.45, 1.3, '(b)', transform=axgr[1].transAxes, fontsize=20)
axgr[2].text(0.45, 1.3, '(c)', transform=axgr[2].transAxes, fontsize=20)
axgr[3].text(0.45, 1.3, '(d)', transform=axgr[3].transAxes, fontsize=20)


counter = 0
i = 0
j = 0
for ax in axgr[0:30]:
    
    if counter%2 == 0:
        

        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.set_title(expt_order_main[i]['label'], fontsize=15)
        data,lons = add_cyclic_point(np.clip(all_co2_Rs[i],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=10, transform=projection, cmap=reversed_map, vmin=0, vmax=1)
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour = contour
        
        i +=1
    else:
        
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        data,lons = add_cyclic_point(np.clip(all_co2_slopes[j],-2,2), coord=lon)

        contour = ax.contourf(lons, lat, data, levels=10, transform=projection, cmap=reversed_map2, vmin=-1, vmax=1, norm=mpl.colors.CenteredNorm())
        #contour = ax.imshow(data, cmap='terrain', vmin=0, vmax=1)
        if i == 3:
            cbar_contour_slope = contour
        j += 1
        
    counter += 1

cb_ax = fig.add_axes([1.02, 0, .017, 0.185])
ticks = [0, .2, .4, .6, .8, 1]
#ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
cbar = fig.colorbar(cbar_contour,cax=cb_ax, orientation='vertical', ticks=ticks)
cbar.ax.tick_params(labelsize=10)
cbar.set_label('R_squared value', fontsize=12)
fig.delaxes(axgr[-1])
fig.delaxes(axgr[-2])





    
    

cb_ax = fig.add_axes([1.09, 0, .017, 0.185])
ticks = [-1,  -.6,  -.3,  0,  .3,  .6, 1, 1.3]
#ticks = [-.3,  -.2,  -.1,  0,  .1,  .2, .3, .4]
cbar = fig.colorbar(cbar_contour_slope,cax=cb_ax, orientation='vertical')#, ticks=ticks)
cbar.ax.tick_params(labelsize=10)
cbar.set_label('slope (ppmv/$^\circ$C)', fontsize=12)
#fig.delaxes(axgr[-1])'''

fig_all = plt.gcf()
plt.show()
fig_all.savefig('/nfs/mary/Users/ee19b2s/work/chapter1/figures/figures_for_manuscript/final_figures_pdfs/fig08_Rs_and_slopes_CO2_vs_SAT_xsmall.pdf', format='pdf', dpi=300, bbox_inches='tight')